<a href="https://colab.research.google.com/github/TusharSharmaji/16_TUSHARSHARMA_D2/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data_dir = "/content/drive/MyDrive/PlantVillage"


In [3]:
import os
print(os.listdir(data_dir))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/PlantVillage'

In [5]:
import os
print(os.listdir("/content/drive/MyDrive"))


['Behavioural_Science[1]_Tushar[1][1].docx', 'Screenshot_2025-02-19-16-14-05-36_40deb401b9ffe8e1df2f1cc5ba480b12.jpg', 'Untitled folder', 'IMG20240819171235 (1).jpg', 'IMG20240819171235.jpg', 'IMG20240819171245.jpg', 'LT 04', 'TUSHAR SHARMA BTECH  SECTION :D ENRL NO : A20405224160', 'DocScanner 28 Apr 2025 10-09\u202fpm.pdf', 'DocScanner 28 Apr 2025 10-35\u202fpm.pdf', 'IMG-20251105-WA0005.jpg', '2025-11-22 00-01-36.pdf', 'oop.xlsx', 'Colab Notebooks', 'Untitled spreadsheet.gsheet', 'How I Built Coding Consistency as a CSE Student - GeeksforGeeks.pdf', 'Tushar_Sharma_One_Page_Resume_2024_2028.pdf', 'Apple.json']


In [6]:
print(os.listdir("/content/drive/MyDrive/Apple.json"))


['SCAB LEAVES', 'APPLE ROT LEAVES', 'HEALTHY LEAVES', 'LEAF BLOTCH']


In [7]:
print(os.listdir(data_dir))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/PlantVillage'

In [10]:
data_dir = "/content/drive/MyDrive/Apple.json/APPLE ROT LEAVES,SCAB LEAVES,HEALTHY LEAVES,LEAF BLOTCH"


In [11]:
print(os.listdir(data_dir))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Apple.json/APPLE ROT LEAVES,SCAB LEAVES,HEALTHY LEAVES,LEAF BLOTCH'

In [13]:
import os

base = "/content/drive/MyDrive"

for root, dirs, files in os.walk(base):
    if "PlantVillage" in dirs:
        print("FOUND:", os.path.join(root, "PlantVillage"))


In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arjuntejaswi/plant-village")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'plant-village' dataset.
Path to dataset files: /kaggle/input/plant-village


In [15]:
import kagglehub

path = kagglehub.dataset_download("arjuntejaswi/plant-village")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'plant-village' dataset.
Path to dataset files: /kaggle/input/plant-village


In [16]:
import os
print(os.listdir(path))

['PlantVillage']


In [17]:
data_dir = os.path.join(path, "PlantVillage")

print(os.listdir(data_dir))


['Pepper__bell___Bacterial_spot', 'Potato___healthy', 'Tomato_Leaf_Mold', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Bacterial_spot', 'Tomato_Septoria_leaf_spot', 'Tomato_healthy', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Potato___Late_blight', 'Tomato_Late_blight', 'Potato___Early_blight', 'Tomato__Tomato_mosaic_virus']


In [18]:
import tensorflow as tf


In [19]:
img_size = 224
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size
)


Found 20638 files belonging to 15 classes.
Using 16511 files for training.
Found 20638 files belonging to 15 classes.
Using 4127 files for validation.


In [20]:
class_names = train_ds.class_names
num_classes = len(class_names)

print("Number of classes:", num_classes)


Number of classes: 15


In [21]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [22]:
model = tf.keras.Sequential([

    tf.keras.layers.Rescaling(1./255),

    base_model,

    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(num_classes, activation='softmax')

])


In [23]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [26]:
epochs = 2

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


Epoch 1/2
516/516 ━━━━━━━━━━━━━━━━━━━━ 839s 2s/step - accuracy: 0.9162 - loss: 0.2597 - val_accuracy: 0.9024 - val_loss: 0.2790
Epoch 2/2
516/516 ━━━━━━━━━━━━━━━━━━━━ 893s 2s/step - accuracy: 0.9344 - loss: 0.1989 - val_accuracy: 0.9009 - val_loss: 0.2823


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("plant_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model created successfully")


In [28]:
  from google.colab import files
files.download("plant_disease_model.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>